<br>

# Introdução

In [ ]:
#!pip install gerador-endereco

In [ ]:
import os
import time
import json
import random
import pandas as pd
import basedosdados as bd
from classify_name import classify_name

<br>

## Lista de Nomes

In [ ]:
# Nomes
df = bd.read_sql(
    """
    SELECT nome FROM `basedosdados.br_cgu_servidores_executivo_federal.servidores_civis_cadastro` LIMIT 100000
    """,
    billing_project_id='inbound-source-328017',
)
display(df.head(5))

# Registros Únicos
list_nomes = list(set(list(df['nome'])))
print(list_nomes[0:5])

<br>

## Lista de Endereços

In [ ]:
from gerador_endereco import *

listas = get_list_ceps_bairros(estado='sp', municipio='piracicaba')
end = get_random_complete_address(random.choice(listas[0]))

<br>

## Tabelas

In [ ]:
i = 1
data = []
while i <= 100:
    small_data = {
        'name': random.choice(list_nomes),
        'address': get_random_complete_address(random.choice(listas[0]))
    }
    data.append(small_data)
    time.sleep(2)
    i += 1

In [ ]:
# Dataframe
df = pd.DataFrame(data)
df.to_csv(
    os.path.join('data', 'nome_endereco.csv'),
    index=False,
)
# Results
display(df.head())


In [ ]:
# Read Dataframe
df = pd.read_csv(os.path.join('data', 'nome_endereco.csv'))

def first_name(name):
    return (name.split(' ')[0]).title()

def last_name(name):
    return (name.split(' ')[-1]).title()

def username(first_name, last_name):
    return '{}{}'.format(first_name.lower(), last_name.lower())

df['first_name'] = df.apply(lambda x: first_name(x['name']), axis=1)
df['last_name'] = df.apply(lambda x: last_name(x['name']), axis=1)
df['username'] = df.apply(lambda x: username(x['first_name'], x['last_name']), axis=1)

df = df.drop_duplicates(subset='username')

display(df.head())
print(len(df))

In [ ]:
# Variables
list_courses = [1, 2, 3, 4, 5, 6, 7, 8, 9]
list_session = [1, 2, 3]

# Parameters
i = 1
pk = 10
data = []

# Loop
for index, row in df.iterrows():
    # Set Variables
    name = row['name']
    address = row['address']

    # dddd
    first_name = (name.split(' ')[0]).title()
    last_name = (name.split(' ')[-1]).title()
    username = '{}{}'.format(first_name.lower(), last_name.lower())
    sexo = classify_name(name)
    course_id = random.choice(list_courses)
    session_year_id = random.choice(list_session)
    if sexo == 'Masculino':
        pic = "/media/menino.jpg"
    elif sexo == 'Feminino':
        pic = "/media/menina.jpg"
    else:
        print('deu ruim!')

    # Parte 1
    item_1 = {
        "model": "core.customuser",
        "pk": pk,
        "fields": {
            "password": "pbkdf2_sha256$260000$Tkf0RF5rJ2MsCoaOBNGl8x$HX1MdI6HNFymYFr+MmISOLkzpmeDUlU5xTLeduRL+z8=",
            "last_login": None,
            "is_superuser": False,
            "username": username,
            "first_name": first_name,
            "last_name": last_name,
            "email": "{}@gmail.com".format(username),
            "is_staff": False,
            "is_active": True,
            "user_type": "3",
            "groups": [],
            "user_permissions": []
        }
    }
    item_2 = {
        "model": "core.students",
        "pk": i,
        "fields": {
            "admin": pk,
            "gender": sexo,
            "profile_pic": pic,
            "address": address,
            "course_id": course_id,
            "session_year_id": session_year_id
        }
    }
    data.append(item_1)
    data.append(item_2)
    i += 1
    pk += 1

print(json.dumps(data, indent=2))

## Save Results

In [ ]:
with open(os.path.join('data', 'students.json'), 'w') as outfile:
    json.dump(data, outfile)